In [1]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

# Covariance Variance VaR Methods

### Build a simple curve for risking and pricing

In [2]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1}),
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.20,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.66,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.93,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 2.20,
}
s_cv = SolvedCurve(nodes=nodes, interpolation="log_linear", swaps=list(swaps.keys()), obj_rates=list(swaps.values()))
s_cv.iterate()  

'tolerance reached (gauss_newton) after 5 iterations, func: 5.269421734819802e-25'

### Add a short historical data sample for covariance

In [3]:
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
historical_chgs.style.format("{:,.1f}")

,2Y,5Y,10Y,30Y
0,-2.9,-3.3,-1.7,-1.6
1,1.8,3.1,1.1,-1.1
2,-0.5,-1.5,0.4,-1.1
3,1.2,0.3,-0.4,1.3
4,4.4,2.0,0.3,-0.9
5,-1.6,-1.6,-2.7,-0.7
6,-1.3,-0.3,-1.4,0.6
7,2.9,2.3,4.0,1.6
8,5.9,5.3,3.2,2.0
9,nan,nan,nan,nan


In [4]:
Q = historical_chgs.cov().to_numpy()
Q

array([[8.66      , 7.3775    , 5.13625   , 2.185     ],
       [7.3775    , 7.3075    , 4.73      , 2.1575    ],
       [5.13625   , 4.73      , 4.89111111, 1.76236111],
       [2.185     , 2.1575    , 1.76236111, 1.86111111]])

### Create a portfolio of swaps replicating the risks in the text

In [5]:
# target = [-10000, 2500, 10000, -4000]
# for i, swap in enumerate(swaps.keys()):
#     swap.notional = (target[i] / swap.analytic_delta(s_cv) * 1e6).real
    
notional = [-50.9e6, 5.23e6, 11.0e6, -1.81e6]
for i, swap in enumerate(swaps.keys()):
    swap.notional = notional[i]

In [6]:
portfolio = Portfolio(objects=list(swaps.keys()))

### Utilise all the CoVaR methods displaying results in table

In [7]:
df = pd.DataFrame(
    index=["2Y", "5Y", "10Y", "30Y"],
    data={
        "S": portfolio.risk(s_cv)[:, 0],
        "S_trade": portfolio.covar_smt(s_cv, Q)[:, 0],
        "c_impact": portfolio.covar_smt_impact(s_cv, Q)[:, 0],
        "%": portfolio.covar_smt_impact(s_cv, Q)[:, 0] / portfolio.covar(s_cv, Q) * 100,
        "VaR alloc": portfolio.covar_alloc(s_cv, Q)[:, 0]
    }
)
df.style.format("{:,.0f}")

fwd tolerance reached (gauss_newton) after 2 iterations, func: 2.2196277897816762e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 3.590051745473591e-26
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.6081619806222394e-26
bck tolerance reached (gauss_newton) after 2 iterations, func: 8.558056137903299e-27
fwd tolerance reached (gauss_newton) after 2 iterations, func: 1.1060470840684078e-25
bck tolerance reached (gauss_newton) after 2 iterations, func: 8.277744275994328e-26
fwd tolerance reached (gauss_newton) after 3 iterations, func: 1.3472659577430203e-26
bck tolerance reached (gauss_newton) after 3 iterations, func: 1.3475617805824782e-26


,S,S_trade,c_impact,%,VaR alloc
2Y,"-10,000","2,942","-2,513",-16,"15,756"
5Y,"2,502","2,297","-1,240",-8,"-2,597"
10Y,"10,006",-484,-36,-0,"1,466"
30Y,"-3,999","3,363",-664,-4,"1,547"


In [8]:
print("Total VaR multiplier: ", portfolio.covar(s_cv, Q))

Total VaR multiplier:  16171.641399973932


### Demonstrate the Multi-Instrument CoVaR methods

In [9]:
combinations = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]
df_mmt = pd.DataFrame(index=["2Y", "5Y", "10Y", "30Y", "impact"])
for combo in combinations:
    df_mmt.loc[["2Y", "5Y", "10Y", "30Y"], f"{combo}"] = portfolio.covar_mmt(s_cv, Q, list(combo))[:, 0]
    df_mmt.loc["impact", f"{combo}"] = portfolio.covar_mmt_impact(s_cv, Q, list(combo))
    
df_mmt.style.format("{:,.0f}")

,"(0, 1)","(0, 2)","(0, 3)","(1, 2)","(1, 3)","(2, 3)"
2Y,"7,042","8,562","2,975",0,0,0
5Y,"-4,812",0,0,"6,979","1,983",0
10Y,0,"-9,476",0,"-7,234",0,"-2,574"
30Y,0,0,-130,0,"1,064","5,800"
impact,"-3,410","-11,600","-2,514","-4,892","-1,286","-1,369"


The specific example of the 2Y and 10Y combination is exemplified in the text.